# Código de geração dos objetos (.obj)

## Bibliotecas

In [1]:
import math
import numpy as np
from perlin_noise import PerlinNoise

## Corpo do fantasma (corpo_fantasma.obj)
Sem olhos e boca

In [2]:
def salvar_esfera_com_cilindro(nome_arquivo, radius=1.0, stacks=10, sectors=20, altura_cilindro=1.0):
    with open(nome_arquivo, 'w') as f:
        f.write("# Meia Esfera com Cilindro abaixo\n")

        vertices = []

        # === MEIA ESFERA ===
        for i in range(stacks + 1):
            stack_angle = math.pi / 2 * (i / stacks)  # de 0 até pi/2
            xy = radius * math.cos(stack_angle)
            z = radius * math.sin(stack_angle)

            for j in range(sectors + 1):
                sector_angle = 2 * math.pi * (j / sectors)
                x = xy * math.cos(sector_angle)
                y = xy * math.sin(sector_angle)

                # Inverte y <-> z
                vertices.append((x, z, y))

                f.write(f"v {x:.6f} {z:.6f} {y:.6f}\n")

        # === CILINDRO ===
        base_index = len(vertices)  # para ajustar índices das faces

        for i in range(2):  # topo e base
            y = 0 if i == 0 else -altura_cilindro
            for j in range(sectors + 1):
                angle = 2 * math.pi * (j / sectors)
                x = radius * math.cos(angle)
                z = radius * math.sin(angle)

                vertices.append((x, y, z))

                f.write(f"v {x:.6f} {y:.6f} {z:.6f}\n")

        # === FACES MEIA ESFERA ===
        for i in range(stacks):
            for j in range(sectors):
                v1 = i * (sectors + 1) + j + 1
                v2 = v1 + sectors + 1
                v3 = v1 + 1
                v4 = v2 + 1

                f.write(f"f {v1} {v2} {v3}\n")
                f.write(f"f {v3} {v2} {v4}\n")

        # === FACES CILINDRO ===
        for j in range(sectors):
            i1 = base_index + j + 1
            i2 = i1 + sectors + 1
            i3 = i1 + 1
            i4 = i2 + 1

            f.write(f"f {i1} {i2} {i3}\n")
            f.write(f"f {i3} {i2} {i4}\n")

    print(f"Arquivo '{nome_arquivo}' gerado com meia esfera + cilindro encaixado!")

In [3]:
salvar_esfera_com_cilindro("corpo_fantasma.obj")

Arquivo 'corpo_fantasma.obj' gerado com meia esfera + cilindro encaixado!


## Árvore (árvore.obj)

In [4]:
def salvar_arvore(nome_arquivo, altura_tronco=0.8, raio_tronco=0.2,
                               altura_copa=2.0, raio_copa=0.6, setores=20):
    with open(nome_arquivo, 'w') as f:
        f.write("# Árvore com cone como copa\n")

        vertices = []

        # === TRONCO (CILINDRO) ===
        for i in range(2):  # base (0) e topo (1)
            y = i * altura_tronco
            for j in range(setores + 1):
                ang = 2 * math.pi * j / setores
                x = raio_tronco * math.cos(ang)
                z = raio_tronco * math.sin(ang)
                vertices.append((x, y, z))
                f.write(f"v {x:.6f} {y:.6f} {z:.6f}\n")

        # === COPA (CONE) ===
        for j in range(setores + 1):
            ang = 2 * math.pi * j / setores
            x = raio_copa * math.cos(ang)
            z = raio_copa * math.sin(ang)
            y = altura_tronco
            vertices.append((x, y, z))
            f.write(f"v {x:.6f} {y:.6f} {z:.6f}\n")

        # topo do cone (pico)
        topo_y = altura_tronco + altura_copa
        vertices.append((0, topo_y, 0))
        f.write(f"v 0.000000 {topo_y:.6f} 0.000000\n")

        # === FACES TRONCO ===
        for j in range(setores):
            v1 = j + 1
            v2 = v1 + setores + 1
            v3 = v1 + 1
            v4 = v2 + 1
            f.write(f"f {v1} {v2} {v3}\n")
            f.write(f"f {v3} {v2} {v4}\n")

        # === FACES COPA ===
        offset = len(vertices) - (setores + 1 + 1)  # início da base da copa
        topo_idx = len(vertices)                   # índice do pico do cone (1-based)
        for j in range(setores):
            base1 = offset + j + 1
            base2 = offset + (j + 1) % setores + 1
            f.write(f"f {base1} {base2} {topo_idx}\n")

    print(f"Arquivo '{nome_arquivo}' gerado com sucesso!")

In [5]:
salvar_arvore("arvore.obj")

Arquivo 'arvore.obj' gerado com sucesso!


## Lua com crateras (lua.obj)

In [6]:
def salvar_lua(nome_arquivo, setores_lua=40, setores_crateras=20, escala_crateras=0.25):
    with open(nome_arquivo, 'w') as f:
        f.write("# Lua com crateras\n")

        vertices = [(0,0,0)]  # centro

        # === LUA (CÍRCULO) ===
        for i in range(setores_lua):
            ang = (2 * math.pi) * (i / setores_lua)
            x = math.cos(ang)
            y = math.sin(ang)
            z = 0.0
            vertices.append((x, y, z))
        for v in vertices:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")
        
        # === FACES LUA ===
        faces = np.ones((setores_lua, 3), dtype=np.int16)
        faces[:, 1] = np.arange(2, len(vertices)+1)
        faces[:, 2] = np.arange(3, len(vertices)+2); faces[-1, 2] = faces[0, 1]
        for face in faces:
            f.write(f"f {face[0]} {face[1]} {face[2]}\n")

        # === Crateras (CÍRCULOS) ===
        num_vertices_lua = len(vertices)
        
        angulos = np.linspace(0, 2 * math.pi, setores_crateras)
        vertices_molde = np.ndarray((setores_crateras+1, 3), dtype=np.float32)
        vertices_molde[0,0:2] = 0  # origem/centro (0,0)
        vertices_molde[1:, 0] = np.vectorize(math.cos)(angulos)
        vertices_molde[1:, 1] = np.vectorize(math.sin)(angulos)
        vertices_molde[:, 2] = -0.05

        f.write("# Cratera 1\n")
        vertices = vertices_molde.copy()

        # ruído Perlin
        noise = PerlinNoise(octaves=5, seed=111)
        noisy_off = np.array([noise([coord[0], coord[1]]) for coord in vertices_molde[1:,0:2]])
        vertices[1:,0] *= (escala_crateras + noisy_off*(escala_crateras**2) )
        vertices[1:,1] *= (escala_crateras + noisy_off*(escala_crateras**2) )

        # posição na lua
        x_off = 0.35
        y_off = 0.2

        vertices[:,0] += x_off
        vertices[:,1] += y_off
        for v in vertices:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")

        # === FACES CRATERA 1 ===
        faces = np.ones((setores_crateras, 3), dtype=np.int16) * (num_vertices_lua+1)
        faces[:, 1] += np.arange(1, len(vertices))
        faces[:, 2] += np.arange(2, len(vertices)+1); faces[-1, 2] = faces[0, 1]
        for face in faces:
            f.write(f"f {face[0]} {face[1]} {face[2]}\n")

        f.write("# Cratera 2\n")
        vertices = vertices_molde.copy()

        # ruído Perlin
        noise = PerlinNoise(octaves=5, seed=222)
        noisy_off = np.array([noise([coord[0], coord[1]]) for coord in vertices_molde[1:,0:2]])
        vertices[1:,0] *= (escala_crateras + noisy_off*(escala_crateras**2) )
        vertices[1:,1] *= (escala_crateras + noisy_off*(escala_crateras**2) )

        # posição na Lua
        x_off = 0.15
        y_off = 0.4

        vertices[:,0] += x_off
        vertices[:,1] += y_off
        for v in vertices:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")

        # === FACES CRATERA 2 ===
        faces = faces.copy() + faces.shape[0] + 1
        for face in faces:
            f.write(f"f {face[0]} {face[1]} {face[2]}\n")

        f.write("# Cratera 3\n")
        vertices = vertices_molde.copy()

        # ruído Perlin
        noise = PerlinNoise(octaves=5, seed=333)
        noisy_off = np.array([noise([coord[0], coord[1]]) for coord in vertices_molde[1:,0:2]])
        vertices[1:,0] *= (escala_crateras + noisy_off*(escala_crateras**2) )
        vertices[1:,1] *= (escala_crateras + noisy_off*(escala_crateras**2) )

        # posição na Lua
        x_off = -0.45
        y_off = -0.35

        vertices[:,0] += x_off
        vertices[:,1] += y_off
        for v in vertices:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")

        # === FACES CRATERA 3 ===
        faces = faces.copy() + faces.shape[0] + 1
        for face in faces:
            f.write(f"f {face[0]} {face[1]} {face[2]}\n")

        f.write("# Cratera 4\n")
        vertices = vertices_molde.copy()

        # ruído Perlin
        noise = PerlinNoise(octaves=5, seed=444)
        noisy_off = np.array([noise([coord[0], coord[1]]) for coord in vertices_molde[1:,0:2]])
        vertices[1:,0] *= (escala_crateras + noisy_off*(escala_crateras**2) )
        vertices[1:,1] *= (escala_crateras + noisy_off*(escala_crateras**2) )

        # posição na Lua
        x_off = 0.3
        y_off = -0.5

        vertices[:,0] += x_off
        vertices[:,1] += y_off
        for v in vertices:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")

        # === FACES CRATERA 4 ===
        faces = faces.copy() + faces.shape[0] + 1
        for face in faces:
            f.write(f"f {face[0]} {face[1]} {face[2]}\n")

    print(f"Arquivo '{nome_arquivo}' gerado com sucesso!")

In [7]:
salvar_lua("lua.obj")

Arquivo 'lua.obj' gerado com sucesso!
